# Building a Bioinformatics Workflow with Jupyter & Anaconda

## Variant Calling Workflow

Burke Squires

https://github.com/burkesquires

---

#### Scenario:

Have yeast NGS data and we want to detect variants from the data

[Source: http://www.htslib.org/workflow/](http://www.htslib.org/workflow/)

Software that is need for pipeline:

- `bwa`
- `samtools`
- `bcftools`

Samtools is a suite of programs for interacting with high-throughput sequencing data. It consists of three separate repositories:

- Samtools: Reading/writing/editing/indexing/viewing SAM/BAM/CRAM format
- BCFtools: Reading/writing BCF2/VCF/gVCF files and calling/filtering/summarising SNP and short indel sequence variants
- HTSlib: A C library for reading/writing high-throughput sequencing data

Samtools and BCFtools both use HTSlib internally, but these source packages contain their own copies of htslib so they can be built independently.

## Download some data:

In [1]:
!curl ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR507/SRR507778/SRR507778_1.fastq.gz | gzip -d | head -100000 > y1.fastq
!curl ftp://ftp.sra.ebi.ac.uk/vol1/fastq/SRR507/SRR507778/SRR507778_2.fastq.gz | gzip -d | head -100000 > y2.fastq
!curl ftp://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/dna/Saccharomyces_cerevisiae.R64-1-1.dna_sm.toplevel.fa.gz | gzip -d > yeast.fasta

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  537M    0  1448    0     0    614      0  10d 15h  0:00:02  10d 15h   614gzip: error writing to output: Broken pipe
  0  537M    0  855k    0     0   284k      0  0:32:13  0:00:03  0:32:10  284k
curl: (23) Failed writing body (0 != 11584)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0  559M    0  328k    0     0   155k      0  1:01:11  0:00:02  1:01:09  155kgzip: error writing to output: Broken pipe
  0  559M    0  902k    0     0   391k      0  0:24:22  0:00:02  0:24:20  391k
curl: (23) Failed writing body (0 != 7240)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3782k  100 3782k    0     0  1361k 

---

## Prepare the BWA indices

We need to ensure there exists a .fai fasta index and also indices for whichever aligner we are using (Bwa-mem in this example).

    samtools faidx yeast.fasta
    bwa index yeast.fasta

In [2]:
!samtools faidx yeast.fasta
!bwa index yeast.fasta

[bwa_index] Pack FASTA... 0.09 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 6.00 seconds elapse.
[bwa_index] Update BWT... 0.06 sec
[bwa_index] Pack forward-only FASTA... 0.05 sec
[bwa_index] Construct SA from BWT and Occ... 1.70 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index yeast.fasta
[main] Real time: 8.029 sec; CPU: 7.927 sec


---

In [3]:
!ls

2 - Bioinformatics software installation using bioconda.ipynb
3 - Variant Calling Pipeline.ipynb
Build-a-Reproducible-Jupyter-Workflow-From-Scratch-master.zip
Hands On.pptx
y1.fastq
y2.fastq
yeast.fasta
yeast.fasta.amb
yeast.fasta.ann
yeast.fasta.bwt
yeast.fasta.fai
yeast.fasta.pac
yeast.fasta.sa


## Produce the Alignments

The aligner is likely to output SAM in the same order or similar order to the input fastq files. It won’t be outputting in chromosome position order, so the output is typically not well suited to CRAM.

    bwa mem -R '@RG\tID:foo\tSM:bar\tLB:library1' yeast.fasta y1.fastq y2.fastq > yeast.sam

The -R option adds a read-group line and applies that read-group to all aligned sequence records. It is not necessary, but a recommended practice.

In [4]:
!bwa mem -R '@RG\tID:foo\tSM:bar\tLB:library1' yeast.fasta y1.fastq y2.fastq > yeast.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 50000 sequences (1800000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (103, 4007, 13219, 91)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (240, 2467, 3114)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 8862)
[M::mem_pestat] mean and std.dev: (1893.02, 1414.42)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 11736)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (243, 275, 314)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (101, 456)
[M::mem_pestat] mean and std.dev: (278.34, 50.30)
[M::mem_pestat] low and high boundaries for proper pairs: (30, 527)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) percentile: (3111, 3320, 3565)
[M::mem_pestat] l

---

## Sort into chromosome/positon order

Ideally at this point we would be outputting CRAM directly, but at present samtools 1.0 does not have a way to indicate the reference on the command line. We can output to BAM instead and convert (below), or modify the SAM @SQ header to include MD5 sums in the M5: field.

    samtools sort -O bam -T /tmp -l 0 -o yeast.bam yeast.sam

The “-l 0” indicates to use no compression in the BAM file, as it is transitory and will be replaced by CRAM soon. We may wish to use -l 1 if disk space is short and we wish to reduce temporary file size.

In [5]:
!samtools sort -O bam -T /tmp -l 0 -o yeast.bam yeast.sam

---

## Convert to CRAM format

    samtools view -T yeast.fasta -C -o yeast.cram yeast.bam

Note that since the BAM file did not have M5 tags for the reference sequences, they are computed by Samtools and added to the CRAM. In a production environment, this step can be avoided by ensuring that the M5 tags are already in the SAM/BAM header.

In [6]:
!samtools view -T yeast.fasta -C -o yeast.cram yeast.bam

__Using CRAM within Samtools__

CRAM is primarily a reference-based compressed format, meaning that only differences between the stored sequences and the reference are stored.

For a workflow this has a few fundamental effects:

Alignments should be kept in chromosome/position sort order.
The reference must be available at all times. Losing it may be equivalent to losing all your read sequences.
Technically CRAM can work with other orders but it can become inefficient due to a large amount of random access across the reference genome. The current implementation of CRAM in htslib 1.0 is also inefficient in size for unsorted data, although this will be rectified in upcoming releases.

In CRAM format the reference sequence is linked to by the md5sum (M5 auxiliary tag) in the CRAM header (@SQ tags). This is mandatory and part of the CRAM specification. In SAM/BAM format, these M5 tags are optional. Therefore converting from SAM/BAM to CRAM requires some additional overhead to link the CRAM to the correct reference sequence.



The last 3 steps can be combined into a pipeline to reduce disk I/O:

    bwa mem yeast.fasta y1.fastq y2.fastq | \
    samtools sort -O bam -l 0 -T /tmp - | \
    samtools view -T yeast.fasta -C -o yeast.cram -

In [7]:
!bwa mem yeast.fasta y1.fastq y2.fastq | \
samtools sort -O bam -l 0 -T /tmp - | \
samtools view -T yeast.fasta -C -o yeast.cram -

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 50000 sequences (1800000 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (103, 4007, 13219, 91)
[M::mem_pestat] analyzing insert size distribution for orientation FF...
[M::mem_pestat] (25, 50, 75) percentile: (240, 2467, 3114)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 8862)
[M::mem_pestat] mean and std.dev: (1893.02, 1414.42)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 11736)
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (243, 275, 314)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (101, 456)
[M::mem_pestat] mean and std.dev: (278.34, 50.30)
[M::mem_pestat] low and high boundaries for proper pairs: (30, 527)
[M::mem_pestat] analyzing insert size distribution for orientation RF...
[M::mem_pestat] (25, 50, 75) percentile: (3111, 3320, 3565)
[M::mem_pestat] l

---

## Viewing in alignment and pileup format

See the variant calling workflow for more advanced examples.

    samtools view yeast.cram
    samtools mpileup -f yeast.fasta yeast.cram

In [8]:
!samtools view yeast.cram > yeast.sam

[W::cram_populate_ref] Creating reference cache directory /Users/squiresrb/.cache/hts-ref
This may become large; see the samtools(1) manual page REF_CACHE discussion


In [9]:
!samtools mpileup -f yeast.fasta yeast.cram > yeast_variants.pileup

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


---

## View (some  of) our results

    head -n 25 yeast.sam
    head -n 25 yeast_variants.pileup

In [10]:
!head -n 25 yeast.sam

SRR507778.19213	147	I	62	60	36M	=	3183	3087	ATCCTAACACTACCCTAACACAGCCCTAATCTAACC	15=@9:@C3<CBGGGDGDBGDFCC?>GGG<GGGDGG	MC:Z:36M	AS:i:36	XS:i:19	MD:Z:36	NM:i:0
SRR507778.12312	147	I	205	60	36M	=	3626	3387	CCACTCACCCACCGTTACCCTCCAATTACCCATATC	GD<B?B>DGGBGGGGGCIIIIGIIIIIEIIGIIIII	MC:Z:36M	AS:i:36	XS:i:26	MD:Z:36	NM:i:0
SRR507778.11604	83	I	402	60	36M	=	3869	3433	CTCACTTGTATACTGATTTTACGTACGCACACGGAT	GHGHIIIIIIIIIIIHIIIIIIIIIIIIIIIIIIII	MC:Z:36M	AS:i:36	XS:i:0	MD:Z:36	NM:i:0
SRR507778.10609	83	I	2661	40	36M	=	6131	3436	TGAATTCGTACAACATTAAACGTGTGTTGGGAGTCG	IIIGFIIGIIHIIIIGIIIIIIIIIIHIIIIIIGII	MC:Z:36M	AS:i:36	XS:i:36	MD:Z:36	NM:i:0
SRR507778.6249	147	I	2925	60	36M	=	6404	3445	TTTCTAAGTGGGATTTTTCTTAATCCTTGGATTCTT	GGGGADIGIHIIHHHIEHIHIHI<DIIIIIIIGIIF	MC:Z:36M	AS:i:36	XS:i:0	MD:Z:36	NM:i:0
SRR507778.14609	129	I	3048	60	36M	IV	1525643	0	AAAAGTAGCCGTTCATTTCCCTTCCGATTTCATTCC	>5833+?=8>B@FBF?9B7AGGGB<G@BGGGGEGD>	MC:Z:36M	AS:i:36	XS:i:0	MD:Z:36	NM:i:0
SRR507778.20233	83	I	3132	60	36M	=	6388	3222	TATT

In [11]:
!head -n 25 yeast_variants.pileup

I	62	a	1	^],	1
I	63	T	1	,	5
I	64	C	1	,	=
I	65	C	1	,	@
I	66	T	1	,	9
I	67	A	1	,	:
I	68	A	1	,	@
I	69	C	1	,	C
I	70	A	1	,	3
I	71	C	1	,	<
I	72	T	1	,	C
I	73	A	1	,	B
I	74	C	1	,	G
I	75	C	1	,	G
I	76	C	1	,	G
I	77	T	1	,	D
I	78	A	1	,	G
I	79	A	1	,	D
I	80	C	1	,	B
I	81	A	1	,	G
I	82	C	1	,	D
I	83	A	1	,	F
I	84	G	1	,	C
I	85	C	1	,	C
I	86	C	1	,	?


---

#### Glossary of file formats

https://www.ncbi.nlm.nih.gov/sra/docs/submitformats/

Sequence data formats:
- FASTA: Simple format for DNA or peptide sequences.
- FASTQ: Stores sequences and sequence __quality__ information together.

Alignment data formats
- SAM / BAM Sequence Alignment/Map

Variation data
- VCF / BCF Variant Call Format